Importing modules

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score
from Layer import *
from network import Network
from activation_func import tanh, tanh_prime, sigmoid, sigmoid_prime, softmax, softmax_prime, relu, relu_prime
from loss_func import mse, mse_prime, cross_entropy, cross_entropy_prime
from data_func import vectorize_labels, k_fold, import_data
from performance_func import plot_error, plot_confusion_matrix
from network_queue import Queue
import time


Importing data

In [2]:
# import data and initialize seed
np.random.seed(10)

# import data
training_size = 3000
normalize = True
training, labels, test, original_test_labels, test_labels = import_data(size=training_size, normalize=normalize)

# intializing queue
queue = Queue(training, labels, test, test_labels, original_test_labels)

#setting up the models

architectures = [[50]]
for layers in architectures:
    queue.add(
        description="architecture: "+str(layers)+" training_size: "+str(training_size),
        netparams={
        "hidden_layers": layers,
        "features": 784,
        "output_classes": 10,
        "activation": relu,
        "activation_prime": relu_prime,
        "loss_activation": softmax,
        "loss_activation_prime": softmax_prime,
        "loss": cross_entropy,
        "loss_prime": cross_entropy_prime
    },
    folds=5,
    params={"epochs": 10,
            "learning_rate": 5e-3,
            "batch_size": 1,
            "momentum": False,
            "weight_decay": 0.01})

#execute queue
results = queue.execute()
np.save("Results/save-"+time.strftime("%Y-%m-%d-%H%M%S"), results)

Fitting model 1/5
epoch 1/10   training error=1.037944  validation error=0.523282 validation accuracy=0.875000 ETA=0:00:31
epoch 2/10   training error=0.420582  validation error=0.473365 validation accuracy=0.855000 ETA=0:00:27
epoch 3/10   training error=0.326298  validation error=0.398174 validation accuracy=0.875000 ETA=0:00:23
epoch 4/10   training error=0.280570  validation error=0.377699 validation accuracy=0.895000 ETA=0:00:19
epoch 5/10   training error=0.254714  validation error=0.355322 validation accuracy=0.901667 ETA=0:00:15
epoch 6/10   training error=0.239113  validation error=0.345420 validation accuracy=0.905000 ETA=0:00:12
epoch 7/10   training error=0.232559  validation error=0.341622 validation accuracy=0.896667 ETA=0:00:09
epoch 8/10   training error=0.224602  validation error=0.333826 validation accuracy=0.906667 ETA=0:00:06
epoch 9/10   training error=0.217410  validation error=0.333075 validation accuracy=0.910000 ETA=0:00:03
epoch 10/10   training error=0.212112

In [7]:
data = np.load("Results/save-2019-04-02-105150.npy")
print(data)

[{'description': 'architecture: [50] training_size: 3000 - Fold 1', 'network': <network.Network object at 0x00000211D14C4BA8>, 'params': {'epochs': 10, 'learning_rate': 0.005, 'batch_size': 1, 'momentum': False, 'weight_decay': 0.01}, 'results': [[1.0379437234654556, 0.42058195769259615, 0.3262979609402861, 0.2805696088713933, 0.2547138542157194, 0.23911286909472962, 0.23255933525134845, 0.2246023172937245, 0.21741021003652258, 0.2121117161156935], [0.5232817382755504, 0.4733654203822177, 0.39817447448317544, 0.37769939981687517, 0.35532181531206947, 0.3454203574592223, 0.34162223872535896, 0.33382614011725176, 0.33307454060072333, 0.3167583850876966], [0.875, 0.855, 0.875, 0.895, 0.9016666666666666, 0.905, 0.8966666666666666, 0.9066666666666666, 0.91, 0.915]], 'accuracies': 0.905}
 {'description': 'architecture: [50] training_size: 3000 - Fold 2', 'network': <network.Network object at 0x00000211D4A41D30>, 'params': {'epochs': 10, 'learning_rate': 0.005, 'batch_size': 1, 'momentum': Fa